In [1]:
import cv2
import os
import sys
import time
import matplotlib.pyplot as plt

import ocr

In [2]:
# 載入YOLOv4模型
from yolo import yolo_v4

 Try to load cfg: ../yolo/weights/yolov4-custom.cfg, weights: ../yolo/weights/yolov4-custom_last.weights, clear = 0 


 0 : compute_capability = 500, cudnn_half = 0, GPU: NVIDIA GeForce MX130 
   layer   filters  size/strd(dil)      input                output
   0 

net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
Create CUDA-stream - 0 
 Create cudnn-handle 0 


conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 route  8 2 	                           ->  208 x 208 x 128 
  10 conv     64       1 x 1/ 1    208 x 208 x 128 ->  208 x 208 x  64 0.709 BF
  11 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
  12 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
 

nms_kind: greedynms (1), beta = 0.600000 


...

nms_kind: greedynms (1), beta = 0.600000 


conv   1024       3 x 3/ 1     13 x  13 x 512 ->   13 x  13 x1024 1.595 BF
 156 conv    512       1 x 1/ 1     13 x  13 x1024 ->   13 x  13 x 512 0.177 BF
 157 conv   1024       3 x 3/ 1     13 x  13 x 512 ->   13 x  13 x1024 1.595 BF
 158 conv    512       1 x 1/ 1     13 x  13 x1024 ->   13 x  13 x 512 0.177 BF
 159 conv   1024       3 x 3/ 1     13 x  13 x 512 ->   13 x  13 x1024 1.595 BF
 160 conv     18       1 x 1/ 1     13 x  13 x1024 ->   13 x  13 x  18 0.006 BF
 161 yolo
[yolo] params: iou loss: ciou (4), iou_norm: 0.07, obj_norm: 1.00, cls_norm: 1.00, delta_norm: 1.00, scale_x_y: 1.05
Total BFLOPS 59.563 
avg_outputs = 489778 
 Allocate additional workspace_size = 6.23 MB 
Loading weights from ../yolo/weights/yolov4-custom_last.weights...

nms_kind: greedynms (1), beta = 0.600000 
 Try to load weights: ../yolo/weights/yolov4-custom_last.weights 

 seen 64, trained: 32 K-images (0 Kilo-batches_64) 


Done! Loaded 162 layers from weights-file 


Loaded - names_list: ../yolo/weights/obj.names, classes = 1 


In [3]:
# 準備影片路徑
video_dir_path = '影片資料集'
video_names_list = os.listdir(video_dir_path)
video_names_list.remove('範例影片.mkv')
video_names_list.sort()

In [4]:
# 如果輸出資料夾不存在則產生
if not os.path.isdir(os.path.join(video_dir_path, 'yolo_out')):
    os.mkdir(os.path.join(video_dir_path, 'yolo_out'))

In [5]:
def id_rule_check(str_id):
    # 檢查第11位編號是否正確
    values = {
        'A': 10, 'B': 12, 'C': 13, 'D': 14, 'E': 15, 'F': 16, 'G': 17, 'H': 18, 'I': 19, 'J': 20,
        'K': 21, 'L': 23, 'M': 24, 'N': 25, 'O': 26, 'P': 27, 'Q': 28, 'R': 29, 'S': 30, 'T': 31,
        'U': 32, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, '1': 1,  '2':  2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9
    }
    total = 0
    for i, letter in enumerate(str_id[:9]):
        if letter in values:
            total += values[letter] * (2 ** i)
    return str(total % 11) == str_id[10]

In [6]:
# 使用迴圈依序讀取影片 進行物件偵測(YOLOv4)和文字辨識(Tesseract-OCR)，並儲存輸出至影片
for video_name in video_names_list:
    video_path = os.path.join(video_dir_path, video_name)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print('Error: Cannot open video', video_path)
        continue

    # 取得輸入影像資訊
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 影片輸出設定
    out = cv2.VideoWriter(os.path.join(video_dir_path, 'yolo_out', f'{video_name}'), cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

    # 儲存辨識結果
    id_text_list = []

    while cap.isOpened():
        print(f'Processing {video_name}: {(int(cap.get(cv2.CAP_PROP_POS_FRAMES))/int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))*100:.2f}%', end='\r')
        start_time = time.perf_counter()
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detections, height_ratio, width_ratio = yolo_v4.image_detector(frame)
        for name, confidence, bbox in detections:
            x_min, y_min, x_max, y_max = yolo_v4.yolo_bbox_2_xyxy(bbox, height_ratio, width_ratio)
            # 裁切號碼區域
            crop_img = frame[y_min:y_max, x_min:x_max]
            # 使用OCR辨識號碼
            try:
                text = ocr.get_text_from_image(crop_img)
            except:
                text = ''
            # 畫號碼區域框
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 20, 255), 3)
            # 文字
            if text != '':
                id_text_list.append(text)
                (textWidth, textHeight), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1 ,2)
                cv2.rectangle(frame, (x_min, y_max), (x_min + textWidth + 2 , y_max + textHeight + 10), (0, 20, 255), -1)
                cv2.putText(frame, text, (x_min+1, y_max+textHeight+5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # 畫fps
        end_time = time.perf_counter()
        fps = 1 / (end_time - start_time)
        (textWidth, textHeight), _ = cv2.getTextSize(f'FPS: {fps:.2f}', cv2.FONT_HERSHEY_SIMPLEX, 2 ,5)
        cv2.rectangle(frame, (30, 80-textHeight-5), (30+textWidth, 80+5), (50, 50, 50), -1)
        cv2.putText(frame, f'FPS: {fps:.2f}', (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 5)

        out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

    print('')
    print(f'Processing {video_name} Done!')
    id_text_list_filtered_len = list(filter(lambda x: len(x)==15, id_text_list))
    if id_text_list_filtered_len != []:
        id_text_list_filtered_len_rule = list(filter(lambda x: id_rule_check(x), id_text_list_filtered_len))
        if id_text_list_filtered_len_rule != []:
            # 超過一筆符合長度與規則要求
            most_common = max(set(id_text_list_filtered_len_rule), key = id_text_list_filtered_len_rule.count)
            confidence = 'high'
        else:
            # 超過一筆符合長度要求
            most_common = max(set(id_text_list_filtered_len), key = id_text_list_filtered_len.count)
            confidence = 'medium'
    else:
        # 都不符合長度要求
        most_common = max(set(id_text_list), key = id_text_list.count)
        confidence = 'low'
    print(f'Most common number: {most_common}, confidence: {confidence};{(id_text_list.count(most_common)/len(id_text_list))*100:.2f}%.')
    video_name_without_ext = os.path.splitext(video_name)[0]
    os.rename(os.path.join(video_dir_path, 'yolo_out', f'{video_name}'),
              os.path.join(video_dir_path, 'yolo_out', f'{video_name_without_ext}_{most_common}.avi'))

    cap.release()
    out.release()

Processing video_0001.avi: 100.00%
Processing video_0001.avi Done!
Most common number: SEKU56753494561, confidence: high;0.95%.
Processing video_0002.avi: 100.00%
Processing video_0002.avi Done!
Most common number: T55U50994004551, confidence: medium;0.39%.
Processing video_0003.avi: 100.00%
Processing video_0003.avi Done!
Most common number: WHSU93278014561, confidence: medium;0.39%.
Processing video_0004.avi: 100.00%
Processing video_0004.avi Done!
Most common number: WHSU16728680EHH, confidence: high;0.33%.
Processing video_0005.avi: 100.00%
Processing video_0005.avi Done!
Most common number: WHSU53681994561, confidence: medium;0.97%.
Processing video_0006.avi: 100.00%
Processing video_0006.avi Done!
Most common number: WHSU1248317PRII, confidence: medium;0.71%.
Processing video_0007.avi: 100.00%
Processing video_0007.avi Done!
Most common number: WHSU516712O4561, confidence: medium;0.30%.
Processing video_0008.avi: 100.00%
Processing video_0008.avi Done!
Most common number: WHSU689